In [ ]:
# STEP 1: INSTALL REQUIRED PACKAGES


# !pip install is a command used in Google Colab (or Jupyter) to install extra libraries
# Libraries are collections of pre-written code made by other developers


# pandas   → helps with handling spreadsheets, tables, and CSV files
# requests → helps in sending/receiving data from websites or APIs
# tabulate → optional library that makes tables look neat when printed
!pip install pandas requests tabulate --quiet
# (--quiet just hides the installation progress messages to keep output clean


# STEP 2: IMPORT LIBRARIES


import os             # 'os' helps with operating system tasks (like working with files and folders)
import json           # 'json' helps us save and read data in JSON format (like a structured text file)
import requests       # 'requests' is used to send requests to APIs (get info from websites)
import pandas as pd   # 'pandas' (imported as pd for short) helps us handle tabular data like Excel/CSV
from tabulate import tabulate  # 'tabulate' helps format data into nice tables for easy reading
from datetime import datetime  # 'datetime' lets us record the current date and time
import numpy as np    # 'numpy' (imported as np) is used for numerical calculations, arrays, math stuff


# STEP 3: FORMATTING TEXT (BOLD HELPER)


# Below, we are defining how to make text appear in BOLD in console output.

BOLD = "\033[1m"   # Special code that tells the console "start bold text"
END = "\033[0m"    # Special code that tells the console "stop bold text"

def bold_caps(text):
    # This function takes in some text,
    # makes it UPPERCASE and BOLD,
    # then returns it.
    return f"{BOLD}{text.upper()}{END}"


# STEP 4: LOGGING HELPER


# "Logs" are like a notebook where the program writes what’s happening step by step.
# They are useful to track errors, progress, or important messages.

logs = []         # This is a list where we will store all the log messages
logs_set = set()  # This is a set (unique collection) to make sure no duplicate logs are stored

def safe_log(text):
    """
    This function saves a log message with the current timestamp,
    but only if the same message has not already been saved before.
    """
    if text not in logs_set:  # if message is not already logged
        # Create a log message with the current time and the text
        logs.append(f"{datetime.now().isoformat()} - {text}")
        # Add text to set so we don’t repeat it again
        logs_set.add(text)


In [ ]:
# STEP 2: LOAD YOUR CSV DATASET FROM GOOGLE DRIVE


# Google Colab allows us to access files stored in Google Drive.
# 'drive.mount' connects your Google Drive to the Colab environment so we can read files from it.

from google.colab import drive
drive.mount('/content/drive')  # Mounts your Google Drive to '/content/drive'

# STEP 2a: SPECIFY PATH TO YOUR CSV FILE


# CSV = Comma-Separated Values, a common format for storing spreadsheet data.
# Replace the path below with the actual location of your CSV file in Google Drive.
csv_path = "/content/drive/MyDrive/dataset/MealsDB.csv"

# STEP 2b: LOAD CSV INTO PANDAS DATAFRAME

# pd.read_csv reads a CSV file and converts it into a pandas DataFrame
# A DataFrame is like a spreadsheet in memory — rows and columns you can easily manipulate.
df_meals = pd.read_csv(csv_path)

# STEP 2c: VERIFY REQUIRED COLUMNS


print("Dataset loaded. Sample entries:")

# 'head()' shows the first 5 rows of the DataFrame to quickly check if the data is loaded correctly
df_meals.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset loaded. Sample entries:


,meal_type,name,calories,protein,fat,carbs,cost,ingredients,recipe
0,breakfast,Oats with Almond Milk 1,300,12,5,50,2.5,Oats;Almond milk;Honey,1. Boil almond milk. 2. Add oats. 3. Cook 5 mi...
1,breakfast,Oats with Almond Milk 2,300,12,5,50,2.5,Oats;Almond milk;Honey,1. Boil almond milk. 2. Add oats. 3. Cook 5 mi...
2,breakfast,Chia Pudding 1,250,8,10,35,3.0,Chia seeds;Almond milk;Honey;Berries,1. Mix chia seeds with almond milk. 2. Refrige...
3,breakfast,Chia Pudding 2,250,8,10,35,3.0,Chia seeds;Almond milk;Honey;Berries,1. Mix chia seeds with almond milk. 2. Refrige...
4,breakfast,Vegetable Omelette 1,300,20,15,10,3.0,Eggs;Spinach;Tomato;Bell pepper,1. Beat eggs. 2. Add chopped vegetables. 3. Co...


In [ ]:
# ------------------------------
# STEP 3: ADMIN AGENT AND MEMORY
# ------------------------------

# This file will store "memory" of user preferences, previous meals, recipes, and feedback.
# Think of it as a small notebook where the program remembers what the user likes or has done before.
memory_file = "meal_memory.json"

# ------------------------------
# STEP 3a: LOAD EXISTING MEMORY OR INITIALIZE NEW MEMORY
# ------------------------------

# If the memory file exists, we load it
if os.path.exists(memory_file):
    with open(memory_file, "r") as f:
        memory = json.load(f)  # load JSON data into a Python dictionary
else:
    # If the file does not exist, create a new memory structure
    memory = {
        "user_preferences": {},  # stores goals, allergies, budget, etc.
        "previous_meals": [],    # stores last 14 days of meals
        "substitutions": {},     # any ingredient substitutions for allergies
        "recipes": {},           # saved recipes
        "editor_feedback": []    # feedback collected from Editor agent
    }


# ------------------------------
# STEP 3b: ADMIN AGENT FUNCTION
# ------------------------------

# The admin_agent() function collects input from the user about their preferences.
def admin_agent():
    print("\n--- ADMIN AGENT (User Input) ---")

    # Ask user for optional notes or query
    query = input("Query/notes (optional): ").strip()

    # Ask for health or diet goals (weight loss, high protein, vegetarian, etc.)
    goals = input("Goals (weight loss/high protein/low fat/vegetarian): ").lower().strip()

    # Ask for allergies (comma-separated list)
    allergies = input("Allergies (comma-separated, e.g., milk, eggs): ").lower().strip()
    # Convert the comma-separated string into a list, remove extra spaces
    allergies = [a.strip() for a in allergies.split(",")] if allergies else []

    # Ask for budget preference
    budget = input("Budget (low/medium/high): ").lower().strip()

    # Save all preferences into the memory dictionary
    memory["user_preferences"] = {
        "query": query,
        "goals": goals,
        "allergies": allergies,
        "budget": budget
    }

    # Log the collected preferences with timestamp
    safe_log(f"Admin Agent: Collected preferences -> Goals: {goals}, Allergies: {allergies}, Budget: {budget}")

    # Return the preferences for use in other parts of the program
    return memory["user_preferences"]


# ------------------------------
# STEP 3c: API KEYS
# ------------------------------

# Prompt user to enter API keys
# CalorieNinjas API key (optional)
calorieninjas_key = input("Enter CalorieNinjas API key (press Enter to skip): ").strip()

# USDA API key (optional)
usda_key = input("Enter USDA API key (press Enter to skip): ").strip()


# ------------------------------
# STEP 3d: FETCH NUTRITION DATA FROM USDA API
# ------------------------------

def fetch_from_usda(food_item, key):
    """
    This function searches USDA database for nutrition info of a food item.
    Returns a dictionary of nutrients, or None if API fails or key is missing.
    """
    if not key:  # if no API key is provided, return None
        return None
    try:
        # Construct the API URL, making sure the food name is URL-safe
        url = f"https://api.nal.usda.gov/fdc/v1/foods/search?query={requests.utils.quote(food_item)}&api_key={key}"
        r = requests.get(url, timeout=8)  # send request to USDA API, wait max 8 seconds

        # If request succeeds
        if r.status_code == 200:
            j = r.json()  # convert response to JSON
            if j.get("foods"):  # check if foods are returned
                fd = j["foods"][0]  # take first food item
                # Extract nutrients from API response
                nutrients = {n.get("nutrientName","").lower(): n.get("value") for n in fd.get("foodNutrients",[])}

                # Return nutrition info as a dictionary
                return {
                    "name": fd.get("description", food_item),
                    "calories": nutrients.get("energy", nutrients.get("energy (kilocalories)", None)),
                    "protein": nutrients.get("protein", None),
                    "fat": nutrients.get("total lipid (fat)", None),
                    "carbs": nutrients.get("carbohydrate, by difference", None),
                    "cost": 0,  # default cost
                    # Combine ingredients into a single string if present
                    "ingredients": ";".join([i.get("name","") for i in fd.get("ingredients",[])] ) if fd.get("ingredients") else ""
                }
    except:
        # If any error occurs (network, parsing, etc.), return None
        return None
    return None


Enter CalorieNinjas API key (press Enter to skip): 4RFKVit47mmOWgW6jDuOTw==wdlz5pzDWgQ9LuLT
Enter USDA API key (press Enter to skip): ghBGWIlQwXgbUihZDaiwOAe91jgDbUP1hHEir8uj


In [ ]:
# ------------------------------
# STEP 4D: EDITOR AGENT (AUTOMATIC CHECKS)
# ------------------------------

# The Editor Agent automatically checks a proposed meal plan
# against user preferences, allergies, recent meals, and basic nutritional goals.

def editor_agent_auto(meal_plan, prefs, prev_meals_flat):
    """
    meal_plan       → dictionary of planned meals (e.g., breakfast, lunch, dinner)
    prefs           → user preferences (goals, allergies, budget)
    prev_meals_flat → list of recently eaten meals to avoid repetition

    Returns:
    feedback → list of issues or suggestions found automatically
    """

    feedback = []  # Will store messages about problems or suggestions

    # Extract user preferences
    goals = prefs.get("goals", "").split()      # Split goals into a list of words
    allergies = prefs.get("allergies", [])      # List of allergens to check
    budget = prefs.get("budget", "")            # Budget preference

    safe_log("Editor: Checking plan...")  # Log start of checks

    # ------------------------------
    # 1. ALLERGEN CHECK
    # ------------------------------
    for mt, info in meal_plan.items():
        meal_name = info["meal"]                    # Meal name
        ingredients = info.get("ingredients", "").lower()  # Ingredients in lowercase for easier comparison
        for allergen in allergies:
            if allergen and (allergen in meal_name.lower() or allergen in ingredients):
                # If allergen is in meal name or ingredients, add to feedback
                feedback.append(f"Issue: Allergen '{allergen}' found in meal '{meal_name}'.")
                safe_log(f"Editor: Flagged allergen '{allergen}' in '{meal_name}'.")

    # ------------------------------
    # 2. RECENT MEAL CHECK (AVOID REPETITION)
    # ------------------------------
    for mt, info in meal_plan.items():
        meal_name = info["meal"]
        if meal_name in prev_meals_flat:
            feedback.append(f"Issue: Meal '{meal_name}' was in recent history.")
            safe_log(f"Editor: Flagged recent meal '{meal_name}'.")

    # ------------------------------
    # 3. BASIC GOAL CHECKS (NUTRITION)
    # ------------------------------
    # Calculate total daily nutrition from all meals
    total_calories = sum([info["nutrition"].get("calories", 0) for info in meal_plan.values() if info["nutrition"]])
    total_protein = sum([info["nutrition"].get("protein", 0) for info in meal_plan.values() if info["nutrition"]])
    total_fat = sum([info["nutrition"].get("fat", 0) for info in meal_plan.values() if info["nutrition"]])
    total_carbs = sum([info["nutrition"].get("carbs", 0) for info in meal_plan.values() if info["nutrition"]])

    # Log total nutrition for reference
    safe_log(f"Editor: Total daily nutrition - Calories: {total_calories}, Protein: {total_protein}, Fat: {total_fat}, Carbs: {total_carbs}")

    # Example goal checks (simple thresholds, can be refined)
    if "high protein" in goals and total_protein < 100:  # Example threshold for high protein goal
        feedback.append(f"Suggestion: Daily protein ({total_protein}g) seems low for high protein goal.")
        safe_log(f"Editor: Suggested higher protein.")

    if "low fat" in goals and total_fat > 50:  # Example threshold for low fat goal
        feedback.append(f"Suggestion: Daily fat ({total_fat}g) seems high for low fat goal.")
        safe_log(f"Editor: Suggested lower fat.")

    if "weight loss" in goals and total_calories > 1500:  # Example threshold for weight loss goal
        feedback.append(f"Suggestion: Daily calories ({total_calories}kcal) might be high for weight loss goal.")
        safe_log(f"Editor: Suggested lower calories for weight loss.")

    # ------------------------------
    # 4. BUDGET CHECK (PLACEHOLDER)
    # ------------------------------
    # Budget check is commented out because cost data and logic need refinement
    # Example: if budget == "low", make sure total cost is under a threshold
    # if budget == "low":
    #     total_cost = sum([info["nutrition"].get("cost", 0) for info in meal_plan.values() if info["nutrition"]])
    #     if total_cost > 10:
    #         feedback.append(f"Suggestion: Daily meal cost (${total_cost:.2f}) might be high for low budget.")

    # ------------------------------
    # 5. FINAL LOG & RETURN
    # ------------------------------
    safe_log(f"Editor: Checks complete. Feedback: {feedback if feedback else 'None'}")
    return feedback  # Return list of issues/suggestions found


In [ ]:
# ------------------------------
# STEP 4A: PLANNER AGENT (TheMealDB → CSV FALLBACK)
# ------------------------------

def planner_agent(forbidden_meals=[]):
    """
    This agent generates meal ideas for breakfast, lunch, and dinner.
    It tries TheMealDB API first for random meals.
    If not enough meals are found, it uses your CSV as a fallback.
    forbidden_meals → list of meals to avoid (flagged by editor)
    """

    prefs = memory["user_preferences"]         # Get user preferences collected by admin agent
    goals = prefs.get("goals","")             # e.g., vegetarian, weight loss
    budget = prefs.get("budget","")           # e.g., low/medium/high

    # Flatten previous meals into a simple list for easier checking
    prev_meals_flat = [m for day in memory.get("previous_meals",[]) for m in day.values()]

    plan_ideas = {}    # Dictionary to store all meal ideas
    selected_set = {}  # Dictionary to store final selected meals
    safe_log("Planner: Generating meal ideas...")  # Log start of planner process

    for mt in ["breakfast","lunch","dinner"]:   # Loop through each meal type
        ideas = []  # Temporary list to store potential meals

        # 1️⃣ Try TheMealDB random meals first
        try:
            for _ in range(10):  # Try up to 10 times to find unique meals
                url = f"https://www.themealdb.com/api/json/v1/1/random.php"
                r = requests.get(url, timeout=8)  # Call the API
                j = r.json()                     # Convert response to JSON
                if j.get("meals"):
                    m = j["meals"][0]            # Take the first meal from results
                    meal_name = m.get("strMeal")
                    # Skip meal if it was eaten recently or in forbidden list
                    if meal_name and meal_name not in prev_meals_flat + forbidden_meals:
                        # Skip non-vegetarian meals if user wants vegetarian
                        if "vegetarian" in goals and any(x in meal_name.lower() for x in ["chicken","fish","egg","beef","salmon"]):
                            continue
                        ideas.append(meal_name)
                        if len(ideas) >= 5:  # Stop if we have 5 ideas
                            break
        except:
            ideas = []  # If API fails, we will use CSV fallback

        # 2️⃣ CSV fallback if not enough ideas
        if len(ideas) < 5:
            pool = df_meals[df_meals["meal_type"]==mt].copy()  # Filter CSV by meal type
            if "vegetarian" in goals:
                # Remove non-vegetarian meals
                pool = pool[~pool["name"].str.lower().str.contains("chicken|fish|egg|salmon|beef")]
            # Remove meals already eaten or forbidden
            pool = pool[~pool["name"].isin(prev_meals_flat + forbidden_meals)]
            # Pick the first few to fill the remaining slots
            csv_ideas = pool.head(5-len(ideas))["name"].tolist()
            ideas.extend(csv_ideas)

        # Choose first idea as final selected meal
        chosen = ideas[0] if ideas else None
        plan_ideas[mt] = {"ideas": ideas, "chosen": chosen}  # Store all ideas and chosen meal
        selected_set[mt] = chosen  # Track final selection
        safe_log(f"Planner: {mt} ideas -> {ideas}")

    safe_log(f"Planner: Selected set -> {selected_set}")
    return plan_ideas, selected_set  # Return all ideas and the final selection


# ------------------------------
# STEP 4B: RESEARCHER AGENT (CalorieNinjas → USDA → OpenFoodFacts → CSV)
# ------------------------------

OFF_SEARCH_URL = "https://world.openfoodfacts.org/cgi/search.pl"

# Function to fetch nutrition info from CalorieNinjas API
def fetch_from_calorieninjas(item, key):
    if not key: return None  # Skip if no API key provided
    try:
        r = requests.get("https://api.calorieninjas.com/v1/nutrition",
                         headers={"X-Api-Key": key},
                         params={"query": item}, timeout=8)
        j = r.json()
        if j.get("items"):
            i = j["items"][0]
            return {"calories": i.get("calories"),
                    "protein": i.get("protein_g"),
                    "fat": i.get("fat_total_g"),
                    "carbs": i.get("carbohydrates_total_g"),
                    "cost": 0,
                    "ingredients": item}
    except: return None
    return None

# Function to fetch nutrition info from OpenFoodFacts
def fetch_from_off(item):
    try:
        r = requests.get(OFF_SEARCH_URL, params={"search_terms": item,
                                                 "search_simple": 1,
                                                 "action": "process",
                                                 "json": 1,
                                                 "page_size": 5}, timeout=8)
        j = r.json()
        if j.get("products"):
            p = j["products"][0]
            nutr = p.get("nutriments", {})
            return {"calories": nutr.get("energy_100g") or nutr.get("energy-kcal_100g"),
                    "protein": nutr.get("proteins_100g"),
                    "fat": nutr.get("fat_100g"),
                    "carbs": nutr.get("carbohydrates_100g"),
                    "cost": 0,
                    "ingredients": p.get("product_name")}
    except: return None
    return None

# Researcher agent collects nutrition info and handles allergy substitutions
def researcher_agent(selected_set):
    meal_plan = {}
    prefs = memory["user_preferences"]
    allergies = prefs.get("allergies",[])

    for mt, meal in selected_set.items():
        # Initialize meal entry
        entry = {"meal": meal, "nutrition": None, "ingredients": None, "alternatives": []}

        #  Try CalorieNinjas
        data = fetch_from_calorieninjas(meal, calorieninjas_key)

        # 2️ Try USDA API if CalorieNinjas fails
        if not data:
            data = fetch_from_usda(meal, usda_key)

        #  Try OpenFoodFacts if previous sources fail
        if not data:
            data = fetch_from_off(meal)

        #  CSV fallback
        if not data:
            row = df_meals[df_meals["name"]==meal]
            if not row.empty:
                r = row.iloc[0]
                data = {"calories": r["calories"], "protein": r["protein"],
                        "fat": r["fat"], "carbs": r["carbs"], "cost": r["cost"],
                        "ingredients": r["ingredients"]}

        # Save nutrition info if available
        if data:
            entry["nutrition"] = {k: np.int64(v) if isinstance(v,float) else v for k,v in data.items() if k!="ingredients"}
            entry["ingredients"] = data.get("ingredients","")

        # Handle allergy substitutions
        SUB_MAP = {"milk":"almond milk","chicken":"tofu","egg":"tofu","fish":"tofu"}
        for a in allergies:
            if a and (a in meal.lower() or (entry["ingredients"] and a in entry["ingredients"].lower())):
                if a in SUB_MAP:
                    alt = meal.lower().replace(a, SUB_MAP[a])
                    entry["alternatives"].append({"reason":f"allergy:{a}", "suggestion": alt})

        meal_plan[mt] = entry

    return meal_plan


# ------------------------------
# STEP 4C: WRITER AGENT (GENERATE RECIPES)
# ------------------------------

def writer_agent(meal_plan):
    """
    Generates recipe text for each meal.
    Uses CSV recipe first, then TheMealDB, then simple fallback recipe.
    """
    safe_log("Writer: Generating recipes...")
    for mt, info in meal_plan.items():
        meal = info["meal"]
        ingredients = info.get("ingredients","")
        recipe_text = ""

        #  Use CSV recipe if available
        row = df_meals[df_meals["name"]==meal]
        if not row.empty and "recipe" in row.columns and pd.notna(row.iloc[0]["recipe"]):
            recipe_text = row.iloc[0]["recipe"]
        else:
            #  TheMealDB API
            try:
                url = f"https://www.themealdb.com/api/json/v1/1/search.php?s={requests.utils.quote(meal)}"
                r = requests.get(url, timeout=8)
                j = r.json()
                if j.get("meals"):
                    m = j["meals"][0]
                    recipe_text = m.get("strInstructions","").strip()
                    # Build ingredient list
                    ing_list = []
                    for i in range(1,21):
                        ing = m.get(f"strIngredient{i}")
                        measure = m.get(f"strMeasure{i}")
                        if ing and ing.strip():
                            ing_list.append(f"{measure.strip() if measure else ''} {ing.strip()}".strip())
                    if ing_list:
                        recipe_text = "Ingredients:\n" + "\n".join([f"- {x}" for x in ing_list]) + "\n\nSteps:\n" + recipe_text
            except:
                recipe_text = ""

            #  Fallback simple recipe if all else fails
            if not recipe_text:
                ing_list = [x.strip() for x in ingredients.split(";") if x.strip()]
                if not ing_list:
                    ing_list = [w for w in meal.replace("with","").replace("and","").split() if w.isalpha()]
                recipe_lines = ["Ingredients:"] + [f"- 100g {x}" for x in ing_list]
                recipe_lines += ["Steps:",
                                 f"1) Prepare ingredients: {', '.join(ing_list) if ing_list else 'main ingredients'}",
                                 "2) Cook main ingredients as required.",
                                 "3) Combine, season, simmer 5-8 mins.",
                                 "4) Serve hot."]
                recipe_text = "\n".join(recipe_lines)

        meal_plan[mt]["recipe"] = recipe_text  # Save recipe in meal plan
        memory["recipes"][meal] = recipe_text  # Save recipe in memory for future use

    return meal_plan




In [ ]:
# ------------------------------
# STEP 5: MAIN INTERACTIVE LOOP
# ------------------------------

# Collect user preferences first using Admin Agent
prefs = admin_agent()  # asks user for goals, allergies, budget, optional notes

session_active = True  # Controls whether we continue planning more days
day_count = 0          # Counter for number of days planned

# Loop until user chooses to exit
while session_active:
    day_count += 1
    print(f"\n***** Planning DAY {day_count} *****")

    # Flatten previous meals from memory into a single list for easy checks
    prev_flat = [m for day in memory.get("previous_meals", []) for m in day.values()]

    regenerate = True     # Flag to control if meal plan needs regeneration
    forbidden_meals = []  # Meals flagged by editor (e.g., allergens, repetition)

    # Inner loop: regenerate meal plan until editor approves
    while regenerate:
        # 1 Planner Agent → generates meal ideas
        plan_ideas, selected_set = planner_agent(forbidden_meals=forbidden_meals)

        # 2Researcher Agent → fetches nutrition info
        meal_plan = researcher_agent(selected_set)

        # 3 Writer Agent → generates recipes
        meal_plan = writer_agent(meal_plan)

        # 4️ Editor Agent → checks for allergens, repeated meals, goals
        feedback = editor_agent_auto(meal_plan, prefs, prev_flat)

        # ------------------------------
        # Structured Output for User
        # ------------------------------
        print("\n====== AGENT FLOW OUTPUT ======")
        # Show admin agent info
        print(f"admin agent: Goals ({prefs.get('goals','')}) Allergies ({','.join(prefs.get('allergies',[]))}) Budget ({prefs.get('budget','')})")

        # Show planner ideas
        for mt, v in plan_ideas.items():
            print(f"[PLANNER] {mt.capitalize()} ideas: {v['ideas']}")

        print("[PLANNER] Selected set:")
        for mt, meal in selected_set.items():
            print(f"- {mt}: {meal}")

        # Show researcher/nutrition info
        for mt, info in meal_plan.items():
            print(f"[RESEARCHER] {mt}: {info['meal']}")
            print(f"  Nutrition: {info['nutrition']}")
            print(f"  Ingredients: {info['ingredients']}")

        # Show writer/recipe info (first 100 chars)
        for mt, info in meal_plan.items():
            print(f"[WRITER] {mt}: Recipe -> {info['recipe'][:100]} ...")

        # Show editor feedback
        print("[EDITOR] Feedback:")
        if feedback:
            for f in feedback:
                print(" -", f)
        else:
            print(" - No issues; plan approved.")
        print("==================================")

        # ------------------------------
        # Regeneration logic if editor flags issues
        # ------------------------------
        if feedback:
            forbidden_meals = []
            for f in feedback:
                # Parse meal names from feedback
                if ":" in f:
                    parts = f.split(":")
                    if len(parts) >= 2:
                        meal_name = parts[1].split("'")[1]
                        forbidden_meals.append(meal_name)
            print("\nEDITOR requests planner to regenerate meals due to mismatched goals/constraints...\n")
        else:
            regenerate = False  # plan approved, exit inner loop

            # Save last 14 days of meals in memory
            memory["previous_meals"].append(selected_set)
            memory["previous_meals"] = memory["previous_meals"][-14:]
            with open(memory_file, "w") as f:
                json.dump(memory, f)  # save memory to JSON file

            # Display final approved meal plan
            print("\nFINAL MEAL PLAN (BOLD CAPS):")
            for mt, meal in selected_set.items():
                print(f"- {mt}: {bold_caps(meal)}")

    # Ask user if they want to plan another day
    next_step = input("\nContinue planning another day? (new meal/exit): ").lower().strip()
    if next_step != "new meal":
        session_active = False  # Exit the loop and end planning



--- ADMIN AGENT (User Input) ---
Query/notes (optional): gym diet
Goals (weight loss/high protein/low fat/vegetarian): weight loss
Allergies (comma-separated, e.g., milk, eggs): milk,eggs
Budget (low/medium/high): low

***** Planning DAY 1 *****

====== AGENT FLOW OUTPUT ======
admin agent: Goals (weight loss) Allergies (milk,eggs) Budget (low)
[PLANNER] Breakfast ideas: ['Paszteciki (Polish Pasties)', 'Mbuzi Choma (Roasted Goat)', 'Vegetarian Casserole', 'Rosol (Polish Chicken Soup)', 'Blackberry Fool']
[PLANNER] Lunch ideas: ['Baingan Bharta', 'Beef Sunday Roast', 'Steak Diane', 'Escovitch Fish', 'Chicken Karaage']
[PLANNER] Dinner ideas: ['Cashew Ghoriba Biscuits', 'Lasagna Sandwiches', 'Nanaimo Bars', 'Timbits', 'Lamb and Potato pie']
[PLANNER] Selected set:
- breakfast: Paszteciki (Polish Pasties)
- lunch: Baingan Bharta
- dinner: Cashew Ghoriba Biscuits
[RESEARCHER] breakfast: Paszteciki (Polish Pasties)
  Nutrition: {'name': 'Polish sausage', 'calories': 337, 'protein': np.int6